# For generating the file name

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Sep  7 20:12:58 2022

@author: COG
"""
import numpy as np
import cv2
import os
import tifffile as tiff

class octFilePath:
    def __init__(self, rawVolumeFilePath, outRootDir = ""):
        
        (self.inRootDir, self.inFileName) = os.path.split(rawVolumeFilePath)
        (self.inFileNameCore, self.inFileNameExt) = os.path.splitext(self.inFileName)

        # Set output root directory.
        if outRootDir == "":
            self.outRootDir = self.inRootDir
        else:
            self.setOutRootDir(outRootDir, append = False)
            # (theDir, theFile) = os.path.split(outRootDir)
            # # Remove "\" at the end of the path
            # if(theFile != ''):
            #     theDir = theDir+theFile
            # self.outRootDir = theDir
            
    def makePath(self, tag = "", ext = ""):
        outPath = self.outRootDir + '\\' + self.inFileNameCore + tag + ext
        return(outPath)
    
    def setOutRootDir(self, outRootDir, append = True):
        """
        outRootDir : string
            The full path of the output folder or subfolder name appended to the input folder to creat the output folder.
        append : Bool
            If true, a subfolder of "outRootDir" is appended to the current output root directory.
            If False, the outRootDir (full path) is set as the output root directory.

        Returns
        -------
        None.

        """
        if append == True:
            self.outRootDir = self.outRootDir + '\\' + outRootDir
            # Make the output sub-directory if it does not exist
            if os.path.exists(self.outRootDir) == False:
                os.makedirs(self.outRootDir)
        else:
            (theDir, theFile) = os.path.split(outRootDir)
            # Remove "\" at the end of the path
            if(theFile != ''):
                theDir = theDir+theFile
            self.outRootDir = theDir


# For making the psudo-color images

In [2]:
import numpy as np
import cv2

def hsv_to_rgb(H, S, V):
        """
        Converts HSL color array to RGB array
    
        H = [0..360]
        S = [0..1]
        V = [0..1]
    
        http://en.wikipedia.org/wiki/HSL_and_HSV#From_HSL
        
        Arguments:
            H: Hue
            S: Saturation
            V: Value (brightness)
    
        Returns:
            R, G, B in [0..255]
        """
        # chroma
        C = V * S
        
        # H' and X (intermediate value for the second largest component)
        Hp = H / 60.0
        X = C * (1 - np.absolute(np.mod(Hp, 2) - 1))
    
        # initilize with zero
        R = np.zeros(H.shape, float)
        G = np.zeros(H.shape, float)
        B = np.zeros(H.shape, float)
    
        # handle each case:
        mask = (Hp >= 0) == ( Hp < 1)
        R[mask] = C[mask]
        G[mask] = X[mask]
    
        mask = (Hp >= 1) == ( Hp < 2)
        R[mask] = X[mask]
        G[mask] = C[mask]
    
        mask = (Hp >= 2) == ( Hp < 3)
        G[mask] = C[mask]
        B[mask] = X[mask]
    
        mask = (Hp >= 3) == ( Hp < 4)
        G[mask] = X[mask]
        B[mask] = C[mask]
    
        mask = (Hp >= 4) == ( Hp < 5)
        R[mask] = X[mask]
        B[mask] = C[mask]
    
        mask = (Hp >= 5) == ( Hp < 6)
        R[mask] = C[mask]
        B[mask] = X[mask]
        
        # adding the same amount to each component, to match value
        m = V - C
        R += m
        G += m
        B += m
        
        # [0..1] to [0..255]
        R *= 255.0
        G *= 255.0
        B *= 255.0
    
        return R.astype(int), G.astype(int), B.astype(int)
    
def hsvToRgbImage(H, S, V):
    R, G, B = hsv_to_rgb(H, S, V)
    temp = np.concatenate([[R], [G], [B]], axis = 0)
    rgbImage = np.rollaxis(temp, 0, len(np.shape(R))+1).astype('uint8')
    return(rgbImage)


def valueRerange(img, inRange, outRange):
    inMin = inRange[0]
    inMax = inRange[1]
    outMin = outRange[0] 
    outMax = outRange[1]

    outImg = np.clip(img, inMin, inMax)
    outImg = ((outImg - inMin) / (inMax - inMin) * (outMax-outMin)) + outMin #120 to make the  maximum color as geren and max SV= 20 and min=0
    return (outImg)


def makePseudoColorImage(H = 0, S = 0, V = 0,
                         inputRanges =  [(43., 70.), (0., 0.), (10., 30.)],
                         outputRanges =[(0., 120.), (0., 0.), (0., 1.)], 
                         blurKernels = [(1,1), (1,1), (1,1)]):

    # Check the H, S, V volumes have the same shape.
    fullChList = [H, S, V]
    volList = []
    i = 0
    npArrayType = type(np.zeros(0))
    for vol in [H, S, V]:
        if type(vol) == npArrayType:
            volList.append(vol)
        i += 1
    
    volShape = volList[0].shape
    for vol in volList:
        if vol.shape != volShape :
            print("makePseudoColorImage: All volumes must have the same shape.")
    
    # Make pseudo color image (volume)
    # Initialize output image matrix (with three channels)
    outImage = np.zeros(volShape + (3,)) # (3ch, x or z size, z or x size)

    thisBscan = np.zeros((3,) + volShape[1:]) # (3ch, x or z size, z or x size)
    for bScanIndex in range(0, volShape[0]):
        chIndex = 0
        for chVol in fullChList:
            if type(chVol) ==  npArrayType:
                # If chVol is a real volume (not a scalar)...
                chBscan = chVol[bScanIndex]
                if blurKernels[chIndex] != (1,1):
                    chBscan = cv2.blur(chBscan, blurKernels[chIndex])
                chBscan = valueRerange(chBscan, inputRanges[chIndex], outputRanges[chIndex])
                thisBscan[chIndex] = chBscan
            else:
                a = valueRerange(chVol, inputRanges[chIndex], outputRanges[chIndex]) # Here chVol is a scalar
                thisBscan[chIndex] = np.ones(volShape[1:]) * a

            chIndex += 1
            
        outImage[bScanIndex] = hsvToRgbImage(thisBscan[0], thisBscan[1], thisBscan[2])
        
    return(outImage)

def makeHVCompiteImage(H, V, 
                       inputRanges = [(10., 30.), (43., 70.)],
                       outputRanges = [(0., 120.), (0., 1.)],
                       blurKernels = [(3,3), (1,1)]):
    outImage = makePseudoColorImage(H = H, S = 1., V = V,
                         inputRanges =  [inputRanges[0],  (0., 1.), inputRanges[1]],
                         outputRanges = [outputRanges[0],  (0., 1.), outputRanges[1]], 
                         blurKernels = [blurKernels[0], (1,1), blurKernels[1]])
    return(outImage)

def makeGrayImage(Vol,
                  inputRange = (43., 70.), outputRange = (0., 1.),
                  blurKernel = (1,1)):
    outImage = makePseudoColorImage(H = 0., S = 0., V = Vol,
                         inputRanges =  [(0., 1.), (0., 1.), inputRange],
                         outputRanges = [(0., 1.), (0., 1.), outputRange], 
                         blurKernels = [(1, 1), (1,1), blurKernel])
    return(outImage)

# Main for the software correction

In [ ]:
# Set the path to the data file and made the file name hander class
# The input data is the multi-frame tiff file at one location (for example, 32 frames in one B-scan location) 
# The data shape is (frame repeat number, pixels in y (axial) , pixels in x (lateral)). The data is in dB scale.
data_path = r"D:\DAta\Skin_09\Example\001_out_20240904_137_OCTIntensityPDavg_dbi_ex_64.tiff"
seq_images = tiff.imread(data_path)
filepath_hander = octFilePath(data_path)

# The frame repeat to match your data, this is for 32 frames per B-scan location
frame_repeat = seq_images.shape[0]
image_size_y = seq_images.shape[1]
image_size_x = seq_images.shape[2]

shiftVal = np.zeros((frame_repeat,2))
DataShifted = np.zeros((frame_repeat,image_size_y,image_size_x))

# For psudo-color LIV generation
liv_min              = 0
liv_max              = 20
oct_min              = -5
oct_max              = 55
inputRanges  = [(liv_min, liv_max), (oct_min, oct_max)]  #
outputRanges = [(0, 120), (0.0, 1.0)]

ref_frame = 15
DataShifted_liv = np.zeros((image_size_y,image_size_x))

start_time = time.time()  # get the start time

# The main loop to calculate the shift value and de-shift the data
for i in range(frame_repeat):
        shiftVal[i], error, diffphase = phase_cross_correlation(seq_images[ref_frame], seq_images[i] ,upsample_factor=10)
        DataShifted[i] = scim.shift(seq_images[i], shiftVal[i])  

end_time = time.time()    # get the end time
duration = end_time - start_time  # calculate difference
print(f"Method run for {duration} s.")

# Save the shifted OCT data
tiff.imwrite(filepath_hander.makePath(tag=f"_corrected_dB", ext=".tiff"), DataShifted)

# Caluculate the LIV and the mean intensity and make the psudo color LIV image
DataShifted_liv = DataShifted.var(axis=0)
DataShifted_mean = np.mean(DataShifted, axis=0)
tiff.imwrite(filepath_hander.makePath(tag=f"_corrected_liv", ext=".tiff"), DataShifted_liv)
pcImage = makeHVCompiteImage(
    H            = DataShifted_liv,
    V            = DataShifted_mean,
    inputRanges  = inputRanges,
    outputRanges = outputRanges,
    blurKernels  = ((1, 1), (1, 1)),
)
tiff.imwrite(
        filepath_hander.makePath(tag=f"_corrected_liv_RGB", ext=".tiff"),
        pcImage.astype("uint8"),
        photometric = "rgb",
)

Method run for 1.84926176071167 s.
